In [1]:
import re
from gurobipy import *
import numpy as np
import itertools

In [2]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrix']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

# Reading in data and running the entire function

In [3]:
filename = r"Data_Xpress_FullDist.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrix'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [5]:
Allocations

array([45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56,
       83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
       90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
       21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
       54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
        1, 33, 59, 42, 52,  0,  0,  0,  0,  0,  0])

In [6]:
def find_closest_product(current_shelf, order, distances):
    closest_shelf = None
    min_distance = float(1e6)
    for shelf in order:
        if distances[current_shelf][shelf] <= min_distance:
            closest_shelf = shelf
            min_distance = distances[current_shelf][shelf]
    return closest_shelf

def generate_order_lists(order):
    ''' 
    Takes a list with tuple elements and returns a list of lists 
    with all possible combinations of individual elements individual elements.
    '''
    order_lists = []
    tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
    for combination in itertools.product(*[order[i] for i in tuple_indices]):
        new_order = order.copy()
        for i, index in enumerate(tuple_indices):
            new_order[index] = combination[i]
        order_lists.append(new_order)
    return order_lists

def greedy_order_route(order, distances):
    ''' 
    Uses a greedy method of calculating the minimum distance. 

    Function has been split into two if statements to consider
    cases of orders where products are contained on more than one shelf.

    If products are contained on more than one shelf, the function constructs 
    a route with all possible shelf combinations and chooses the one with 
    the shortest distance.

    Returns a list containing the route and the total distance for the order.
    '''

    # If all products in the order are contained on one shelf only
    if not any(isinstance(product, tuple) for product in order):
        visited = [0]
        current_position = 0  
        for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
        visited.append(0)
        OrderDistance = 0
        for i in range(len(visited) - 1):
            OrderDistance += distances[visited[i]][visited[i+1]]
        order_distance_final = OrderDistance
        visited_final = visited

    # If one or more products in the order are contained on more than one shelf
    elif any(isinstance(product, tuple) for product in order):
        order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
        order_routes = []   # initialise a list of routes for all combinations                                 
        order_distances = [] # initialise a list of distances for all combinations
        
        # loop over all combinations
        for order in order_combinations:
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
            visited.append(0)
            order_routes.append(visited) # add the route for the combination to the list of routes
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
        # select the order with the shortest distance among the combinations
        min_idx = order_distances.index(min(order_distances))
        visited_final = order_routes[min_idx] 
        order_distance_final = order_distances[min_idx]
            
    return visited_final, order_distance_final # return order route and distance

def convert_orders_to_shelf_indices(allocations):
    ''' 
    This function takes the allocation vector and returns an 
    order matrix with shelf indices instead of product indices.
    '''
    product_to_shelf = {}
    for shelf_index, product in enumerate(allocations):
        if product != 0:  # Check if the element is not zero
            if product not in product_to_shelf:
                product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
            else:
                # If the product already exists in the dictionary, append the new shelf index to the list
                product_to_shelf[product].append(shelf_index + 1)

    # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
    product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

    OrdersByShelf = []
    for order in Orders:
        order_shelf_indices = []
        for product_index in order:
            if product_index in product_to_shelf_tuples:
                shelf_indices = product_to_shelf_tuples[product_index]
                order_shelf_indices.append(shelf_indices)
            else:
                order_shelf_indices.append(0)  # Product not found in allocation matrix
        OrdersByShelf.append(order_shelf_indices)

    return OrdersByShelf

In [7]:
def q1_function(allocation_vector):   

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    Routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, FullDistanceMatrix)
        Routes.append(visited_order_route)
        DistancesPerOrder.append(3*visited_order_dist)
        TotalDistance += 3*visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse = True)
    
    # List of routes ordered by distance (descending)
    OrderedRoutes = [Routes[i] for i in idx_longest_orders]

    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes

In [8]:
### TEST CASE ON CURRENT ALLOCATION ###
TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes = q1_function(Allocations)

#### Obtaining total distances for different allocations

In [9]:
print(f"Distance for the current allocation: {q1_function(Allocations)[0]} metres.")
print(f"Distance for all shelves filled: {q1_function(allocations_full)[0]} metres.")
print(f"Distance for some repeated products and some empty shelves: {q1_function(allocations_mid_full)[0]} metres.")

Distance for the current allocation: 290706 metres.
Distance for all shelves filled: 289200 metres.
Distance for some repeated products and some empty shelves: 289728 metres.


## The functions contained within the distance function (for testing and sanity checks)

In [10]:
# def find_closest_product(current_position, order, distances):
#     closest_shelf = None
#     min_distance = float(1e6)
#     for shelf in order:
#         if distances[current_position][shelf] <= min_distance:
#             closest_shelf = shelf
#             min_distance = distances[current_position][shelf]
#     return closest_shelf

# def generate_order_lists(order):
#     ''' 
#     Takes a list with tuple elements and returns a list of lists 
#     with all possible combinations of individual elements individual elements.
#     '''
#     order_lists = []
#     tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
#     for combination in itertools.product(*[order[i] for i in tuple_indices]):
#         new_order = order.copy()
#         for i, index in enumerate(tuple_indices):
#             new_order[index] = combination[i]
#         order_lists.append(new_order)
#     return order_lists

# def greedy_order_route(order, distances):
#     ''' 
#     Uses a greedy method of calculating the minimum distance. 

#     Function has been split into two if statements to consider
#     cases of orders where products are contained on more than one shelf.

#     If products are contained on more than one shelf, the function constructs 
#     a route with all possible shelf combinations and chooses the one with 
#     the shortest distance.

#     Returns a list containing the route and the total distance for the order.
#     '''

#     # If all products in the order are contained on one shelf only
#     if not any(isinstance(product, tuple) for product in order):
#         visited = [0]
#         current_position = 0  
#         for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#         visited.append(0)
#         OrderDistance = 0
#         for i in range(len(visited) - 1):
#             OrderDistance += distances[visited[i]][visited[i+1]]
#         order_distance_final = OrderDistance
#         visited_final = visited

#     # If one or more products in the order are contained on more than one shelf
#     elif any(isinstance(product, tuple) for product in order):
#         order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
#         order_routes = []   # initialise a list of routes for all combinations                                 
#         order_distances = [] # initialise a list of distances for all combinations
        
#         # loop over all combinations
#         for order in order_combinations:
#             visited = [0]
#             current_position = 0  
#             for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#             visited.append(0)
#             order_routes.append(visited) # add the route for the combination to the list of routes
#             OrderDistance = 0
#             for i in range(len(visited) - 1):
#                 OrderDistance += distances[visited[i]][visited[i+1]]
#             order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
#         # select the order with the shortest distance among the combinations
#         min_idx = order_distances.index(min(order_distances))
#         visited_final = order_routes[min_idx] 
#         order_distance_final = order_distances[min_idx]
            
#     return visited_final, order_distance_final # return order route and distance

# def convert_orders_to_shelf_indices(allocations):
#     ''' 
#     This function takes the allocation vector and returns an 
#     order matrix with shelf indices instead of product indices.
#     '''
#     product_to_shelf = {}
#     for shelf_index, product in enumerate(allocations):
#         if product != 0:  # Check if the element is not zero
#             if product not in product_to_shelf:
#                 product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
#             else:
#                 # If the product already exists in the dictionary, append the new shelf index to the list
#                 product_to_shelf[product].append(shelf_index + 1)

#     # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
#     product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

#     OrdersByShelf = []
#     for order in Orders:
#         order_shelf_indices = []
#         for product_index in order:
#             if product_index in product_to_shelf_tuples:
#                 shelf_indices = product_to_shelf_tuples[product_index]
#                 order_shelf_indices.append(shelf_indices)
#             else:
#                 order_shelf_indices.append(0)  # Product not found in allocation matrix
#         OrdersByShelf.append(order_shelf_indices)

#     return OrdersByShelf



# allocation_vector = allocations_full # change this to test different allocations

# OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector) 
# TotalDistance = 0           # initialise counter for total distance
# DistancesPerOrder = []      # initalise list to contain the distances for each order 
# routes = []                 # intialise list to contain the routes for each order
# for order in OrdersByShelf:
#     visited_order_route, visited_order_dist = greedy_order_route(order, FullDistanceMatrix)
#     routes.append(visited_order_route)
#     DistancesPerOrder.append(3*visited_order_dist)
#     TotalDistance += 3*visited_order_dist

# # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
# SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

# # The indices corresponding to the longest orders (descending order) 
# idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

# Checking Construction Heuristics

In [14]:
from collections import Counter

In [23]:
def getMostCommonProducts(Orders):
    #flattens list and removes 0
    flattenedList = [x for x in [item for sublist in Orders for item in sublist] if x != 0]

    #gets counts for products
    itemCounts = Counter(flattenedList)

    #sorts and returns only original number, not the count
    sorted_items = sorted(itemCounts.items(), key=lambda x: x[1], reverse=True)
    original_numbers = [item for item, _ in sorted_items]

    return itemCounts, original_numbers

def getClosestShelves(Distances):
    #Gets the final row of the matrix as this represents the distance to door (packaging)
    distanceToDoor = Distances[-1]

    #gets the index, and distance
    indexed_numbers = list(enumerate(distanceToDoor))

    #sorts based on distance and then takes only the index
    sorted_indexed_numbers = sorted(indexed_numbers, key=lambda x: x[1])
    sorted_indexes = [index for index, _ in sorted_indexed_numbers]

    #ignores the first value as this is the door -> door distance = 0
    return sorted_indexes

def ConstructionHeuristic_mostCommonOrder(Orders, Distances):
    mostCommonProducts = getMostCommonProducts(Orders)

    ClosestShelves = getClosestShelves(Distances)

    newShelves = [0]*96

    for i, prod in enumerate(mostCommonProducts):

        newShelves[i] = prod

    return newShelves, ClosestShelves

In [28]:
# OrdersByShelf = convert_orders_to_shelf_indices()
# shelves = ConstructionHeuristic_mostCommonOrder()
getClosestShelves(FullDistanceMatrix)

[96,
 71,
 95,
 70,
 94,
 69,
 93,
 68,
 92,
 24,
 48,
 67,
 91,
 23,
 47,
 66,
 90,
 22,
 46,
 65,
 89,
 21,
 45,
 64,
 88,
 20,
 44,
 63,
 87,
 19,
 43,
 62,
 86,
 18,
 42,
 61,
 85,
 17,
 41,
 60,
 84,
 16,
 40,
 59,
 83,
 15,
 39,
 58,
 82,
 14,
 38,
 57,
 81,
 13,
 37,
 56,
 80,
 12,
 36,
 55,
 79,
 11,
 35,
 54,
 78,
 10,
 34,
 53,
 77,
 9,
 33,
 52,
 76,
 8,
 32,
 51,
 75,
 7,
 31,
 50,
 74,
 6,
 30,
 49,
 73,
 5,
 29,
 4,
 28,
 0,
 3,
 27,
 2,
 26,
 1,
 25]